In [1]:
import pandas as pd
import numpy as np
from folktables import ACSDataSource, ACSEmployment, ACSIncomePovertyRatio, ACSMobility, ACSIncome, ACSHealthInsurance, ACSPublicCoverage, ACSTravelTime, generate_categories
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
import os


In [25]:
path = './acs_west_mobility/dataset_ref.csv'

In [26]:
df = pd.read_csv(path)

In [28]:
df.reset_index(inplace=True)

In [29]:
df

,index,AGEP,WKHP,JWMNP,PINCP,SCHL_1.0,SCHL_2.0,SCHL_3.0,SCHL_4.0,SCHL_5.0,...,COW_8.0,COW_9.0,ESR_1.0,ESR_2.0,ESR_3.0,ESR_4.0,ESR_5.0,ESR_6.0,RAC1P,MIG
0,0,-0.366312,-1.640392,-0.823844,-0.719153,1,0,0,0,0,...,0,0,0,0,0,0,0,1,0,1
1,1,0.937985,-1.640392,-0.823844,-0.719153,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,1
2,2,-1.235843,-0.553046,-0.378654,-0.541672,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,1
3,3,1.372750,0.534299,-0.823844,-0.442559,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,1
4,4,-1.670608,-1.640392,-0.823844,-0.714543,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
81948,81948,-1.670608,-1.640392,-0.823844,-0.719153,0,0,0,0,0,...,0,1,0,0,1,0,0,0,0,1
81949,81949,0.720602,0.316830,0.066536,0.548566,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,1
81950,81950,-0.801077,-0.009374,0.511726,0.387220,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
81951,81951,0.068454,0.534299,-0.823844,0.018429,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,1


In [52]:
data_source = ACSDataSource(
    survey_year='2018', horizon='1-Year', survey='person')

west_states = ["CA", "OR", "WA", "NV", "AZ"]
east_states = ['ME', 'NH', 'MA', 'RI', 'CT', 'NY',
                'NJ', 'DE', 'MD', 'VA', 'NC', 'SC', 'GA', 'FL']
acs_data = data_source.get_data(states=west_states, download=True)

acss = ['acs_west_poverty', 'acs_west_mobility', 'acs_west_income', 'acs_west_insurance',
        'acs_west_public', 'acs_west_travel', 'acs_west_poverty', 'acs_west_employment']
param = [128, 4, 8, 16, 32, 64, 128, 256]
folks = [ACSIncomePovertyRatio, ACSMobility, ACSIncome, ACSHealthInsurance,
            ACSPublicCoverage, ACSTravelTime, ACSIncomePovertyRatio, ACSEmployment]




acs_task, task_name, seed = folks[0], acss[0], param[0]
#pprint(inspect.getmembers(acs_task))
group_var = acs_task.group
target_var = acs_task.target
groups_to_keep = [1, 2]
acs_data = acs_data.loc[acs_data[group_var].isin(groups_to_keep)]
dataX, dataY, dataA = acs_task.df_to_pandas(acs_data)
# taking catoegorical features
definition_df = data_source.get_definitions(download=True)
categories = generate_categories(features=acs_task.group, definition_df=definition_df)
categories_cols = categories.keys()
# taking non-categorical features
# non_categorical_cols = [col for col in dataX.columns if col not in categories_cols]
# dataX[non_categorical_cols] = dataX[non_categorical_cols].astype(float)
# # use standard scaler to scale non_categorical_cols
# pipe = Pipeline([('scaler', StandardScaler())])
# dataX[non_categorical_cols] = pipe.fit_transform(dataX[non_categorical_cols])
# # use one-hot encoding to encode categorical_cols
# dataX = pd.get_dummies(dataX, columns=categories_cols)
# # process dataA and dataY
# columnsY = dataY.columns
# columnsA = dataA.columns
# #categorize dataA
# dataA = dataA.apply(lambda x: x.astype('category').cat.codes)
# dataY = dataY.apply(lambda x: x.astype('category').cat.codes)
# df = create_dataset_ref(dataX, dataA, dataY)
# # # create folder with task_name if not exist
# if not os.path.exists('{}'.format(task_name)):
#     os.makedirs('{}'.format(task_name))
# path = '{}/dataset_ref.csv'.format(task_name)
# df.to_csv(path, index=False)